In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline\\steps'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig_SVR:
    root_dir: Path
    model_dir_name: str
    X_train_data_path : Path
    y_train_data_path : Path
    X_test_data_path : Path
    y_test_data_path : Path
    model_name : str
    x_scaler_name : str
    y_scaler_name : str
    c : float
    gamma : float
    runtime_name : str

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config_SVR(self) -> ModelTrainerConfig_SVR:
        config = self.config.model_trainer_svr
        params = self.params.SVR
        
        create_directories([config.root_dir])
        create_directories([os.path.join(config.root_dir,config.model_dir_name)])
        model_trainer_config = ModelTrainerConfig_SVR(
            root_dir = config.root_dir,
            model_dir_name = config.model_dir_name,
            X_train_data_path=config.X_train_data_path,
            y_train_data_path=config.y_train_data_path,
            X_test_data_path= config.X_test_data_path,
            y_test_data_path= config.y_test_data_path,
            model_name = config.model_name,
            x_scaler_name = config.x_scaler_name,
            y_scaler_name = config.y_scaler_name,
            c = params.c,
            gamma = params.gamma,
            runtime_name = config.runtime_name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.mlProject.utils import logger
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
import time

In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig_SVR):
        self.config = config
        
    def load_scale_data(self):
        X_train =np.array(pd.read_csv(self.config.X_train_data_path))
        y_train =np.array(pd.read_csv(self.config.y_train_data_path))
        X_test = np.array(pd.read_csv(self.config.X_test_data_path))
        y_test = np.array(pd.read_csv(self.config.y_test_data_path))
        
        std_x = StandardScaler()
        X_train_scaled = std_x.fit_transform(X_train)
        X_test_scaled = std_x.transform(X_test)

        std_y = StandardScaler()
        y_train_scaled = std_y.fit_transform(y_train)
        y_test_scaled = std_y.transform(y_test)
        
        np.savetxt(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),"X_test_scaled.csv"), X_test_scaled, delimiter=',')
        np.savetxt(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),"y_test_scaled.csv"), y_test_scaled, delimiter=',')
        
        joblib.dump(std_x,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.x_scaler_name))
        joblib.dump(std_y,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.y_scaler_name))

        
        return X_train_scaled,y_train_scaled
        
    def train(self,X_train_scaled,y_train_scaled):    
        svr = SVR(kernel='rbf', C=self.config.c, gamma=self.config.gamma)
        multioutput_svr = MultiOutputRegressor(svr)

        start_svr = time.time()
        multioutput_svr.fit(X_train_scaled, y_train_scaled)
        end_svr = time.time()
        
        training_time = end_svr - start_svr
        training_time = {"training time" : training_time}
        save_json(path=Path(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.runtime_name)),data=training_time)        
        joblib.dump(multioutput_svr,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config_SVR()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    X_train_scaled,y_train_scaled = model_trainer_config.load_scale_data()
    model_trainer_config.train(X_train_scaled,y_train_scaled)
except Exception as e:
    raise e

[2024-04-17 15:34:05,701: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-17 15:34:05,708: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-17 15:34:05,714: INFO: common: created directory at: artifacts]
[2024-04-17 15:34:05,718: INFO: common: created directory at: artifacts/model_trainer]
[2024-04-17 15:34:05,721: INFO: common: created directory at: artifacts/model_trainer\SVR]
[2024-04-17 15:34:09,390: INFO: common: json file saved at: artifacts\model_trainer\SVR\lr_runtime.json]
